In [9]:

import tensorflow as tf
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings(action="ignore")


data=pd.read_csv("./data/bmi/bmi.csv" , sep="," , skiprows = 3) #sep 는 csv가 ,로 구분되어있어서 defalt 값??
       #결치값 제거

data = data.dropna(how="any")

df_x = data[["height","weight"]]
df_y = data["label"]
df_x2=df_x

# df_y 를 분류해야함 100  010 001 , 0 1 2

y_data=tf.one_hot(df_y , 3)

sess=tf.Session()
y_data = sess.run(y_data)

scaler= MinMaxScaler()
x_data = scaler.fit_transform(df_x.values)


X = tf.placeholder(shape=[None,2],dtype = tf.float32  ) #행 숫자는 무상관
Y = tf.placeholder(shape=[None,3],dtype = tf.float32  )

W= tf.Variable(tf.random_normal([2,3]), name="weight"  )
b= tf.Variable(tf.random_normal([3]),name="bias")


logit = tf.matmul(X,W)+b
H=tf.nn.softmax(logit)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2( logits=logit , labels=Y ))

train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())



for step in range(10000):
    _,cost_val = sess.run([train,cost], feed_dict={X:x_data,Y:y_data} )
    if step%1000 == 0 :
        print("cost:{}".format(cost_val))

        
predict=tf.argmax(H , 1)  
correct=tf.equal(predict , tf.argmax(Y,1)) 

accuracy=tf.reduce_mean(tf.cast(correct , dtype=tf.float32))

print("정확도={}".format(sess.run(accuracy, feed_dict={X:x_data,Y:y_data} )) )



x1=int(input('height :'))
x2=int(input('weight : '))

np=np.array([[x1,x2]])

x_data2 = scaler.transform(np) 

result=sess.run(predict, feed_dict={X:x_data2})


if result[0]==0:
    print("thin")
elif result[0]==1:
    print("normal")
else:
    print("fat")


cost:1.6860613822937012
cost:0.510639488697052
cost:0.41103002429008484
cost:0.3597971796989441
cost:0.3263689875602722
cost:0.30214110016822815
cost:0.2834547758102417
cost:0.26843130588531494
cost:0.2559868097305298
cost:0.24544385075569153
정확도=0.9803500175476074
height :180
weight : 55
thin


In [2]:
# import tensorflow as tf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import warnings

tf.reset_default_graph()

train = pd.read_csv("./data/mnist2/train.csv")
test = pd.read_csv("./data/mnist2/test.csv")


x_data=train.drop("label", axis=1, inplace=False)
train2=train["label"]



nn=int(len(x_data))

X=tf.placeholder(shape=[None,784], dtype=tf.float32)
Y=tf.placeholder(shape=[None,10], dtype=tf.float32)

keep_rate=tf.placeholder(dtype=tf.float32)  #드랍아웃에서 얼마나 살릴지


df1=pd.DataFrame()
arr = np.zeros((10,10), dtype=np.int32)

df1=arr

df2=pd.DataFrame()


for n in range(0,10):
    df1[n,n]=1
    df2[n]=np.arange(10)

tr2=int(len(train2))


for n in range(0,tr2):
    df2[n]=df1[train2[n]]

y_data=pd.DataFrame()

y_data=df2.transpose()


number=int(len(x_data)*2/3)

x_traindata=x_data[0:number]
x_testdata=x_data[number:-1]

y_traindata=y_data[0:number]
y_testdata=y_data[number:-1]



X_img = tf.reshape(X, shape=[-1,28,28,1])

W1=tf.Variable(tf.random_normal([5,5,1,32],stddev=0.01))

L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1], padding="SAME")

L1 = tf.nn.relu(L1)
p1 = tf.nn.max_pool(L1 , ksize=[1,2,2,1] , strides=[1,2,2,1], padding="SAME")

L2=tf.layers.conv2d(inputs=p1, filters=64, kernel_size=[5,5] , padding="SAME", strides=1 ,activation=tf.nn.relu)



L2=tf.layers.max_pooling2d(inputs=L2 , pool_size=[2,2], padding="SAME" , strides=2)

#########CONVOLUTION LAYER 끝

#4. NURAL NETWORK

L2=tf.reshape(L2,shape=[-1,7*7*64])

#print(L2.shape)

# 5. weight & bias

W2=tf.get_variable("weight2" , shape=[7*7*64,256], initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.random_normal([256]),name="bias2" )

_layer1 = tf.nn.relu(tf.matmul(L2,W2)+b2 )
layer1 = tf.nn.dropout(_layer1 , keep_prob=keep_rate)


W3=tf.get_variable("weight3" , shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.random_normal([256]),name="bias3" )

_layer2 = tf.nn.relu(tf.matmul(layer1,W3)+b3 )
layer2 = tf.nn.dropout(_layer2 , keep_prob=keep_rate)


W4=tf.get_variable("weight4" , shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.random_normal([10]),name="bias4" )

# hypothesis 가설만들어야함


H = tf.matmul(layer2,W4)+b4

#cost 함수

cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=H , labels=Y))

train = tf.train.AdamOptimizer(learning_rate=0.000833).minimize(cost)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

#학습
num_of_epoch = 10
batch_size=100

for step in range(10):
    num_of_iter = 42000
    print(step)
    n=0
    n1=100
    n2=100
    
    for i in range(280):
        batch_x = x_traindata[n:n2]
        batch_y = y_traindata[n:n2]
        _,cost_val=sess.run([train,cost], feed_dict={X:batch_x , Y:batch_y ,keep_rate:0.5
                                                   })
        
        n=n+n1
        n2=n2+n1
    print("cost : {}".format(cost_val))
    

predict = tf.argmax(H,1)
correct=tf.equal(predict , tf.argmax(Y,1)) 
accuracy=tf.reduce_sum(tf.cast(correct , dtype=tf.float32))

###테스트

result_sum=0
num_of_iter = int(num_of_iter/100)

n=0
n1=100
n2=100

for i in range(140):
    batch_x=x_testdata[n:n2]
    batch_y =y_testdata[n:n2]
    accuracy_num=sess.run(accuracy, feed_dict={X:batch_x , Y:batch_y ,keep_rate:1})
    result_sum += accuracy_num
    n=n+n1
    n2=n2+n1

print("정확도:{}".format(result_sum/14000))


0
cost : 0.20990148186683655
1
cost : 0.17270921170711517
2
cost : 0.02657024934887886
3
cost : 0.028033185750246048
4
cost : 0.057378824800252914
5
cost : 0.03582988679409027
6
cost : 0.086913101375103
7
cost : 0.029871154576539993
8
cost : 0.025502528995275497
9
cost : 0.0035948692820966244
정확도:0.9880714285714286
